# Project: App-Review Miner
Team members: Shanshan Li, Yingyezhe Jin, Tianshu Chu, Xiao Huang

In [7]:
# Put all import here
import numpy as np
import lda
import lda.datasets

### NLP based preprocessing

    Inputs:  raw_data        
    Outputs: AllR      is an n*m np matrix, n is the number of total reviews, m is the length of vocabulary
             vocab     is the corresponding n vocabulary list

In [9]:
# Code here

### Naive Bayes based filtering

    Inputs:  AllR      is an n*m np matrix, n is the number of total reviews, m is the length of vocabulary
    Outputs: InfoR     is a k*m np matrix, k is the number of informative reviews, m is the length of vocabulary

In [10]:
# Code here

### LDA topic clustering

    Inputs:  InfoR    is the k*m np matrix, k is the number of informative reviews, m is the length of vocabulary
             vocab    is the corresponding n vocabulary list
             n_topics is the number of topics
    Outputs: doc_topi is a k*n_topics np matrix, which indicates the probability

In [14]:
# Inputs here
InfoM = lda.datasets.load_reuters()
vocab = lda.datasets.load_reuters_vocab()
n_topics = 20
# Code here
model = lda.LDA(n_topics, n_iter=1500, random_state=1)
model.fit(InfoM) 
topic_word = model.topic_word_
n_top_words = 8
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))
doc_topi = model.doc_topic_

Topic 0: british churchill sale million major letters west britain
Topic 1: church government political country state people party against
Topic 2: elvis king fans presley life concert young death
Topic 3: yeltsin russian russia president kremlin moscow michael operation
Topic 4: pope vatican paul john surgery hospital pontiff rome
Topic 5: family funeral police miami versace cunanan city service
Topic 6: simpson former years court president wife south church
Topic 7: order mother successor election nuns church nirmala head
Topic 8: charles prince diana royal king queen parker bowles
Topic 9: film french france against bardot paris poster animal
Topic 10: germany german war nazi letter christian book jews
Topic 11: east peace prize award timor quebec belo leader
Topic 12: n't life show told very love television father
Topic 13: years year time last church world people say
Topic 14: mother teresa heart calcutta charity nun hospital missionaries
Topic 15: city salonika capital buddhist c

In [20]:
doc_topic = model.doc_topic_
print doc_topic[1]

[ 0.09492754  0.1673913   0.00072464  0.00072464  0.00072464  0.14565217
  0.00072464  0.05144928  0.00072464  0.00072464  0.00797101  0.03695652
  0.00072464  0.25434783  0.00072464  0.00072464  0.00072464  0.01521739
  0.00797101  0.21086957]


### Ranking all the groups based on importance

    Inputs:  
    Outputs: 

### Ranking all the reviews in each group based on text rank

    Inputs:  
    Outputs: 

In [ ]:
# Code here